# Boston housing price regression

In [1]:
# import modules 
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import keras
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


## Import data from keras
First step is to import boston housing data from kears.
<br>The format is in array.
<br>Training set and test set are separated based on seed(optional).
<br>Input housing features and output housing prices are also separated.
<br>Further information about the usage, please visit https://keras.io/datasets/

In [2]:
# import boston housing data from keras
(x_train, y_train), (x_test, y_test) = keras.datasets.boston_housing.load_data(seed=42)

In [3]:
x_train

array([[9.1780e-02, 0.0000e+00, 4.0500e+00, ..., 1.6600e+01, 3.9550e+02,
        9.0400e+00],
       [5.6440e-02, 4.0000e+01, 6.4100e+00, ..., 1.7600e+01, 3.9690e+02,
        3.5300e+00],
       [1.0574e-01, 0.0000e+00, 2.7740e+01, ..., 2.0100e+01, 3.9011e+02,
        1.8070e+01],
       ...,
       [3.0410e-02, 0.0000e+00, 5.1900e+00, ..., 2.0200e+01, 3.9481e+02,
        1.0560e+01],
       [5.2058e-01, 0.0000e+00, 6.2000e+00, ..., 1.7400e+01, 3.8845e+02,
        9.5400e+00],
       [2.5199e-01, 0.0000e+00, 1.0590e+01, ..., 1.8600e+01, 3.8943e+02,
        1.8060e+01]])

## Normalize input features
Because the input features are in different scales, it is necessary to normalize the input features so that they share the same scales
<br>sklearn.preprocessing.StandardScaler is used to normalize
<br>DO NOT mix with sklearn.preprocessing.normalize

In [4]:
# implement StandardScaler to normalize features
scaler = StandardScaler()
# use fit_transform to obtain mean and variance for x_train (fit) and then normalize x_train (transform)
x_train = scaler.fit_transform(x_train)
# use transform to normalize x_test 
x_test = scaler.transform(x_test)

## Separate validation set from original training set 
Use sklearn.model_selection.train_test_split to separate 
<br>Assign test_size or train_size 

In [5]:
# separate validation set from original training set
# use train_test_split
x_train, x_valid, y_train, y_valid = train_test_split(x_train,y_train, test_size=0.2, random_state=42)

## Build neural network
Use a function to build this neural network with three hidden layers
<br>This function use the numbers of neurons in each hidden layer as parameter (in total three input parameters)

In [6]:
# build neural network

def boston_model(n_first,n_second,n_third):
    model = Sequential()
    
    # input layer
    model.add(Dense(n_first,activation='relu',input_dim=x_train.shape[1])) 
    # input_shape can also be used to input the shape for input data 
    # model.add(Dense(n_first,activation='relu',input_shape = (x_train.shape[1],)))
    
    # hidden layers
        # second layer
    model.add(Dense(n_second,activation='relu')) # no need to specify input_dim in hidden layers
        # third layer
    model.add(Dense(n_third, activation='relu'))
    
    # output layer
    model.add(Dense(1)) # Since default activation function is linear/identy, no need to define
    # compilation
    model.compile(optimizer='adam',loss='mae')
    return model

## Hyperparameter selection
Use dataframe to store the mae for validation under different combinations of neurons 
<br>This dataframe will then be used to find the best case and then use this combination to predict the outcome for test set 

First is to create a dataframe to store result

In [7]:
result = pd.DataFrame(columns=['first_hidden','second_hidden','third_hidden','mae'])

In [8]:
result.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 4 columns):
first_hidden     0 non-null object
second_hidden    0 non-null object
third_hidden     0 non-null object
mae              0 non-null object
dtypes: object(4)
memory usage: 0.0+ bytes


Second is to train the model and get the mae from validation set 
<br>Parameters to be tested for each layer: 
<br>First layer: 20,30
<br>Second layer: 12,18
<br>Third layer: 3,10
<br>In total 8 combinations
<br><br>For each combination, epochs =30

In [9]:
# use for loop to run each combination
for first_layer in [20,30]:
    for second_layer in [12,18]:
        for third_layer in [3,10]:
            model = boston_model(first_layer,second_layer,third_layer)
            # train the neural network, validation set is included to check if overfitting occurs
            model.fit(x_train, y_train, epochs=50, validation_data=(x_valid, y_valid))
            # get the performance by using validaiton set 
            mae = model.evaluate(x_valid, y_valid)
            # record the details in the result dataframe 
            # use append() function 
            #result = result.append({'first_hidden':first_layer,'second_hidden':second_layer,'third_hidden':third_layer,'mae':mae}, ignore_index=True)
            result=result.append({'first_hidden':first_layer,'second_hidden':second_layer,'third_hidden':third_layer,'mae':mae},ignore_index=True)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Train on 323 samples, validate on 81 samples
Epoch 1/50
323/323 [==============================] - 1s 3ms/step - loss: 22.4274 - val_loss: 22.3159
Epoch 2/50
323/323 [==============================] - 0s 65us/step - loss: 22.2478 - val_loss: 22.1425
Epoch 3/50
323/323 [==============================] - 0s 49us/step - loss: 22.0488 - val_loss: 21.9382
Epoch 4/50
323/323 [==============================] - 0s 83us/step - loss: 21.8144 - val_loss: 21.6912
Epoch 5/50
323/323 [==============================] - 0s 59us/step - loss: 21.5288 - val_loss: 21.3620
Epoch 6/50
323/323 [==============================] - 0s 65us/step - loss: 21.1452 - val_loss: 20.9317
Epoch 7/50
323/323 [==============================] - 0s 62us/step - loss: 20.6439 - val_loss: 20.3634
Epoch 8/50
323/323 [==============================] - 0s 62us/step - loss: 19.9915 - val_loss: 19.6404
Epoch 9/50
3

323/323 [==============================] - 0s 80us/step - loss: 4.1013 - val_loss: 4.0487
Epoch 25/50
323/323 [==============================] - 0s 71us/step - loss: 3.9681 - val_loss: 3.9082
Epoch 26/50
323/323 [==============================] - 0s 71us/step - loss: 3.8658 - val_loss: 3.7317
Epoch 27/50
323/323 [==============================] - 0s 71us/step - loss: 3.7623 - val_loss: 3.6205
Epoch 28/50
323/323 [==============================] - 0s 62us/step - loss: 3.6587 - val_loss: 3.5068
Epoch 29/50
323/323 [==============================] - 0s 68us/step - loss: 3.5836 - val_loss: 3.4236
Epoch 30/50
323/323 [==============================] - 0s 83us/step - loss: 3.5031 - val_loss: 3.3763
Epoch 31/50
323/323 [==============================] - 0s 86us/step - loss: 3.4462 - val_loss: 3.3212
Epoch 32/50
323/323 [==============================] - 0s 71us/step - loss: 3.3875 - val_loss: 3.2625
Epoch 33/50
323/323 [==============================] - 0s 74us/step - loss: 3.3350 - val_loss:

323/323 [==============================] - 0s 65us/step - loss: 21.0802 - val_loss: 20.7726
Epoch 3/50
323/323 [==============================] - 0s 65us/step - loss: 20.2781 - val_loss: 19.9371
Epoch 4/50
323/323 [==============================] - 0s 52us/step - loss: 19.2485 - val_loss: 18.8590
Epoch 5/50
323/323 [==============================] - 0s 59us/step - loss: 18.0135 - val_loss: 17.6076
Epoch 6/50
323/323 [==============================] - 0s 59us/step - loss: 16.5345 - val_loss: 16.1372
Epoch 7/50
323/323 [==============================] - 0s 52us/step - loss: 14.9200 - val_loss: 14.5775
Epoch 8/50
323/323 [==============================] - 0s 53us/step - loss: 13.3221 - val_loss: 13.0072
Epoch 9/50
323/323 [==============================] - 0s 59us/step - loss: 11.6550 - val_loss: 11.4525
Epoch 10/50
323/323 [==============================] - 0s 62us/step - loss: 9.8665 - val_loss: 9.9645
Epoch 11/50
323/323 [==============================] - 0s 65us/step - loss: 8.3340 - 

323/323 [==============================] - 0s 56us/step - loss: 3.5908 - val_loss: 3.4946
Epoch 32/50
323/323 [==============================] - 0s 62us/step - loss: 3.5097 - val_loss: 3.4327
Epoch 33/50
323/323 [==============================] - 0s 52us/step - loss: 3.4757 - val_loss: 3.3274
Epoch 34/50
323/323 [==============================] - 0s 74us/step - loss: 3.4342 - val_loss: 3.2953
Epoch 35/50
323/323 [==============================] - 0s 52us/step - loss: 3.3960 - val_loss: 3.2489
Epoch 36/50
323/323 [==============================] - 0s 62us/step - loss: 3.3425 - val_loss: 3.1661
Epoch 37/50
323/323 [==============================] - 0s 52us/step - loss: 3.3669 - val_loss: 3.1509
Epoch 38/50
323/323 [==============================] - 0s 62us/step - loss: 3.3031 - val_loss: 3.0340
Epoch 39/50
323/323 [==============================] - 0s 52us/step - loss: 3.2140 - val_loss: 3.0423
Epoch 40/50
323/323 [==============================] - 0s 56us/step - loss: 3.1963 - val_loss:

323/323 [==============================] - 0s 56us/step - loss: 18.4387 - val_loss: 17.7951
Epoch 10/50
323/323 [==============================] - 0s 56us/step - loss: 16.9056 - val_loss: 16.1147
Epoch 11/50
323/323 [==============================] - 0s 56us/step - loss: 14.9453 - val_loss: 13.9858
Epoch 12/50
323/323 [==============================] - 0s 65us/step - loss: 12.6036 - val_loss: 12.0762
Epoch 13/50
323/323 [==============================] - 0s 56us/step - loss: 10.6408 - val_loss: 10.7064
Epoch 14/50
323/323 [==============================] - 0s 59us/step - loss: 9.3408 - val_loss: 9.7067
Epoch 15/50
323/323 [==============================] - 0s 90us/step - loss: 8.5282 - val_loss: 8.9319
Epoch 16/50
323/323 [==============================] - 0s 59us/step - loss: 7.9796 - val_loss: 8.2389
Epoch 17/50
323/323 [==============================] - 0s 65us/step - loss: 7.4262 - val_loss: 7.7066
Epoch 18/50
323/323 [==============================] - 0s 71us/step - loss: 6.9140 -

323/323 [==============================] - 0s 56us/step - loss: 3.0212 - val_loss: 2.6966
Epoch 39/50
323/323 [==============================] - 0s 59us/step - loss: 2.9724 - val_loss: 2.6394
Epoch 40/50
323/323 [==============================] - 0s 56us/step - loss: 2.9378 - val_loss: 2.5885
Epoch 41/50
323/323 [==============================] - 0s 56us/step - loss: 2.9059 - val_loss: 2.5551
Epoch 42/50
323/323 [==============================] - 0s 71us/step - loss: 2.8696 - val_loss: 2.5589
Epoch 43/50
323/323 [==============================] - 0s 59us/step - loss: 2.8455 - val_loss: 2.5378
Epoch 44/50
323/323 [==============================] - 0s 59us/step - loss: 2.8004 - val_loss: 2.5378
Epoch 45/50
323/323 [==============================] - 0s 59us/step - loss: 2.7801 - val_loss: 2.5518
Epoch 46/50
323/323 [==============================] - 0s 62us/step - loss: 2.7546 - val_loss: 2.4784
Epoch 47/50
323/323 [==============================] - 0s 52us/step - loss: 2.7276 - val_loss:

## Result

In [10]:
result

,first_hidden,second_hidden,third_hidden,mae
0,20.0,12.0,3.0,2.562792
1,20.0,12.0,10.0,2.335937
2,20.0,18.0,3.0,2.474858
3,20.0,18.0,10.0,2.169455
4,30.0,12.0,3.0,2.538287
5,30.0,12.0,10.0,2.150151
6,30.0,18.0,3.0,2.355836
7,30.0,18.0,10.0,2.373490


Then we obtain the hyperparameters for the best combination by minimizing mae

In [11]:
# first is to know which combination provides the minimized mae 
# use idxmin() function to check 
min_index = result['mae'].idxmin()
# will return the index number 
print(min_index)

5


In [12]:
# then we obtain the numbers for the three hidden layers 

In [13]:
first_hidden_best = result['first_hidden'][min_index]
second_hidden_best = result['second_hidden'][min_index]
third_hidden_best = result['third_hidden'][min_index]

print("Optimized number for the first layer is ",first_hidden_best)
print("Optimized number for the second layer is ",second_hidden_best)
print("Optimized number for the third layer is ",third_hidden_best)

Optimized number for the first layer is  30.0
Optimized number for the second layer is  12.0
Optimized number for the third layer is  10.0


## Predict the test set 

In [14]:
# first, change the hidden_best from float back to integer
first_hidden_best  = int(first_hidden_best)
second_hidden_best = int(second_hidden_best)
third_hidden_best  = int(third_hidden_best)

# then train the model again
min_model = boston_model(first_hidden_best,second_hidden_best,third_hidden_best)
min_model.fit(x_train, y_train, epochs=50, validation_data=(x_valid, y_valid))
min_mae = min_model.evaluate(x_test, y_test)

print("Mean absolute Error for the test set: ", min_mae)

Train on 323 samples, validate on 81 samples
Epoch 1/50
323/323 [==============================] - 1s 3ms/step - loss: 22.8974 - val_loss: 22.5999
Epoch 2/50
323/323 [==============================] - 0s 62us/step - loss: 22.5691 - val_loss: 22.3543
Epoch 3/50
323/323 [==============================] - 0s 62us/step - loss: 22.3393 - val_loss: 22.1549
Epoch 4/50
323/323 [==============================] - 0s 59us/step - loss: 22.1206 - val_loss: 21.9173
Epoch 5/50
323/323 [==============================] - 0s 74us/step - loss: 21.8428 - val_loss: 21.5821
Epoch 6/50
323/323 [==============================] - 0s 56us/step - loss: 21.4313 - val_loss: 21.0610
Epoch 7/50
323/323 [==============================] - 0s 65us/step - loss: 20.7996 - val_loss: 20.2461
Epoch 8/50
323/323 [==============================] - 0s 68us/step - loss: 19.8181 - val_loss: 19.0575
Epoch 9/50
323/323 [==============================] - 0s 62us/step - loss: 18.4074 - val_loss: 17.4522
Epoch 10/50
323/323 [========